In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install tensorflow -U tensorflow_decision_forests > /dev/null

In [2]:
import pandas as pd
import numpy as np
from numpy import dtype
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
cols = {
    
    # 'ts': int,
    # 'src_ip': 'category',
    'src_port': float,
    # 'dst_ip': 'category',
    'dst_port': float,
    'proto': 'category',
    # 'service': 'category',
    # 'duration': float,
    # 'src_bytes': float,
    # 'dst_bytes': float,
    # 'conn_state': 'category',
    # 'missed_bytes': float,

    # 'src_pkts': float,
    # 'src_ip_bytes': float,
    # 'dst_pkts': float,
    # 'dst_ip_bytes': float,

    # 'dns_query': 'category',
    # 'dns_qclass': float,
    # 'dns_qtype': float,
    # 'dns_rcode': float,
    # 'dns_AA': float, #bool
    # 'dns_RD': float, #bool
    # 'dns_rejected': float, #bool

    # 'ssl_version': 'category',
    # 'ssl_cipher': 'category',
    # 'ssl_resumed': float, #bool
    # 'ssl_established': float, #bool
    # 'ssl_subject': 'category',
    # 'ssl_issuer': 'category',

    # 'http_trans_depth': float,
    # 'http_method': 'category',
    # 'http_uri': 'category',
    # 'http_version': 'category',
    # 'http_request_body_len': float,
    # 'http_response_body_len': float,
    # 'http_status_code': float,
    # 'http_user_agent': 'category',
    # 'http_orig_mime_types': 'category',
    # 'http_resp_mime_types': 'category',

    # 'weird_name': 'category',
    # 'weird_addl': 'category',
    # 'weird_notice': float, #bool

    'label': int

}

def bool_to_number(x: str) -> float:
  if x == 'T':
    return 1
  elif x == 'F':
    return 0
  elif x == '-':
    return 0.5
  else:
    raise x

converters = {
    'dns_AA': bool_to_number,
    'dns_RD': bool_to_number,
    'dns_rejected': bool_to_number,
    'ssl_resumed': bool_to_number,
    'ssl_established': bool_to_number,
    'weird_notice': bool_to_number
}

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/datasci/ton_iot/Train_Test_Network.csv',
                      sep=',',
                      header=0,
                      dtype=cols,
                      usecols=list(cols.keys()),
                      converters=converters,
                      na_values=['-'])

In [ ]:
df

,src_port,dst_port,proto,label
0,1883.0,52976.0,tcp,0
1,47260.0,15600.0,udp,0
2,1880.0,51782.0,tcp,0
3,34296.0,10502.0,tcp,0
4,46608.0,53.0,udp,0
...,...,...,...,...
461038,33108.0,80.0,tcp,1
461039,37242.0,443.0,tcp,1
461040,4444.0,49178.0,tcp,1
461041,60816.0,443.0,tcp,1


In [ ]:
features = list(df.columns)
features.remove('label')

In [ ]:
for col,typ in dict(df.dtypes).items():
  if typ == 'category':
    df[col] = df[col].cat.codes.astype(int)
  else:
    vs = list(df[col].unique())
    # should not happen in the full dataset, otherwise would be removed in 1st step of preproc
    if (len(vs) == 1) and (np.isnan(vs[0])):
      print('error',col,'const column with NaN. Replacing with 0')
      df[col] = 0
    elif (len(vs) == 2) and (np.isnan(vs[0]) or np.isnan(vs[1])):
      val = vs[1] if np.isnan(vs[0]) else vs[0]
      val = -val if val != 0 else 1
      df[col] = df[col].fillna(value=val)
    else:
      df[col] = df[col].fillna(df[col].mean())

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel()
model.compile()

Use /tmp/tmpsktygu_v as temporary training directory


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('label',axis=1).values, 
                                                    df['label'].values, 
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
history = model.fit(X_train,
                    Y_train,
                    verbose = 1)

Reading training dataset...
Training dataset read in 0:00:09.413021. Found 322730 examples.
Training model...
Model trained in 0:01:34.853241
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [ ]:
predict = model.predict(X_test, verbose=1)

4323/4323 [==============================] - 7s 2ms/step


In [ ]:
tp,tn,fp,fn = 0,0,0,0
predictn = predict.flatten().round().tolist()
len(predictn)
for i in range(len(predictn)):
  if predictn[i]==1 and Y_test[i]==1:
    tp+=1
  elif predictn[i]==0 and Y_test[i]==0:
    tn+=1
  elif predictn[i]==0 and Y_test[i]==1:
    fp+=1
  elif predictn[i]==1 and Y_test[i]==0:
    fn+=1
    
print(tp,tn,fp,fn)
classification_report(Y_test, predictn, output_dict=True)

44994 88188 3347 1784


{'0': {'f1-score': 0.9717311178081286,
  'precision': 0.9634347517343093,
  'recall': 0.9801716089450051,
  'support': 89972},
 '1': {'f1-score': 0.9460570443339396,
  'precision': 0.9618624139552782,
  'recall': 0.9307627066051591,
  'support': 48341},
 'accuracy': 0.9629029809200871,
 'macro avg': {'f1-score': 0.958894081071034,
  'precision': 0.9626485828447937,
  'recall': 0.9554671577750822,
  'support': 138313},
 'weighted avg': {'f1-score': 0.9627579165485524,
  'precision': 0.962885212785865,
  'recall': 0.9629029809200871,
  'support': 138313}}

In [ ]:
print(model.summary())

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (3):
	data:0.0
	data:0.1
	data:0.2

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.  "__LABEL"  4.884305 ################
    2. "data:0.2"  3.041888 #######
    3. "data:0.1"  2.087467 ###
    4. "data:0.0"  1.360542 

Variable Importance: NUM_AS_ROOT:
    1. "data:0.0" 129.000000 ################
    2. "data:0.2" 87.000000 #
    3. "data:0.1" 83.000000 

Variable Importance: NUM_NODES:
    1. "data:0.0" 4785.000000 ################
    2. "data:0.1" 2866.000000 ########
    3. "data:0.2" 511.000000 

Variable Importance: SUM_SCORE:
    1. "data:0.1" 164486.059187 ################
    2. "data:0.2" 905

In [ ]:
model.save('/content/drive/Shareddrives/datasci/ton_iot/GBT.tf')
with open('/content/drive/Shareddrives/datasci/ton_iot/result/GBT.csv','w') as infile:
  txt = ''
  for pred in predictn:
    txt += str(int(pred))+','
  txt = txt[:-1]
  txt += '\n'
  infile.write(txt)

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/datasci/ton_iot/GBT.tf/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/datasci/ton_iot/GBT.tf/assets
